In [1]:
import pandas as pd
from pandas import read_parquet
import os
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
%matplotlib inline

In [2]:
import astropy.units as u
from astropy.coordinates import SkyCoord
def match(df_1, df_2, pixel, df1_name):
    """
    match two catalog
    :param df_1:
    :param df_2:
    :return:
    """
    sdss = SkyCoord(ra=df_1.ra * u.degree, dec=df_1.dec * u.degree)
    decals = SkyCoord(ra=df_2.ra * u.degree, dec=df_2.dec * u.degree)
    idx, d2d, d3d = sdss.match_to_catalog_sky(decals)
    max_sep = pixel * 0.262 * u.arcsec
    distance_idx = d2d < max_sep
    sdss_matches = df_1.iloc[distance_idx]
    matches = idx[distance_idx]
    decal_matches = df_2.iloc[matches]
    test = sdss_matches.loc[:].rename(columns={"ra":"%s" % df1_name[0], "dec":"%s" % df1_name[1]})
    test.insert(0, 'ID', range(len(test)))
    decal_matches.insert(0, 'ID', range(len(decal_matches)))
    new_df = pd.merge(test, decal_matches, how="inner", on=["ID"])
    return new_df.drop("ID", axis=1)

In [3]:
# https://doi.org/10.5281/zenodo.4573248
# arXiv:2102.08414v2
# here is Mike Walmsley's catalog
auto_catalog_path = "/data/renhaoye/MorCG/dataset/gz_decals_auto_posteriors.parquet"
df_auto = read_parquet(auto_catalog_path).rename(columns=lambda x: x.replace("-", "_")) # 头中的-换为_

In [4]:
# main galaxy sample raw catalog, 74w sample
with fits.open("/data/renhaoye/MorCG/dataset/VAGC_MGS-m14_1777--20180116.fits") as hdul:
    ra = np.array(hdul[1].data["RA      "])
    dec = np.array(hdul[1].data["DEC     "])
    petro_mag = np.array(hdul[1].data["PETROMAG"][:, 2])
    # z = np.array(hdul[1].data["Z       "])
    MGS = pd.DataFrame(np.array((ra, dec, petro_mag)).T, columns=["ra", "dec", "petro_mag"])

In [8]:
from tqdm import  tqdm
# load local file names to pd.Dataframe to match
local_files = os.listdir("/data/renhaoye/MorCG/dataset/in_decals/raw_fits/")
ra, dec = [], []
for i in tqdm(range(len(local_files))):
    if ".fits" in local_files[i]:
        t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1].split(".fits")[0])
        ra.append(t_ra)
        dec.append(t_dec)
decals_new = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

100%|███████████████████████████████████████████████████████████████████████| 361930/361930 [00:00<00:00, 476475.26it/s]


In [9]:
# load local file names to pd.Dataframe to match
local_files = os.listdir("/data/renhaoye/MorCG/dataset/out_decals/raw_fits/")
ra, dec = [], []
for i in tqdm(range(len(local_files))):
    if ".fits" in local_files[i]:
        t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1].split(".fits")[0])
        ra.append(t_ra)
        dec.append(t_dec)
out_decals_new = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

100%|███████████████████████████████████████████████████████████████████████| 297843/297843 [00:00<00:00, 493975.81it/s]


In [14]:
test = match(decals_new, out_decals_new, 2, df1_name=["in_ra", "in_dec"])

In [15]:
test

,in_ra,in_dec,ra,dec
0,156.404678,32.128263,156.404678,32.128263
1,4.391916,0.347087,4.391916,0.347087
2,174.082308,34.291213,174.082308,34.291213
3,202.998092,34.065763,202.998092,34.065763
4,224.084632,33.953941,224.084632,33.953941
...,...,...,...,...
62492,230.296703,32.045519,230.296703,32.045519
62493,134.507709,32.576229,134.507709,32.576229
62494,226.149239,34.727260,226.149239,34.727260
62495,221.380369,34.762659,221.380369,34.762659


In [ ]:

# load local file names to pd.Dataframe to match
local_files = os.listdir("/data/renhaoye/MorCG/dataset/in_decals/raw_fits/")
ra, dec = [], []
for i in tqdm(range(len(local_files))):
    if ".fits" in local_files[i]:
        t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1])
        ra.append(t_ra)
        dec.append(t_dec)
decals_new = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

In [5]:
from tqdm import  tqdm
# load local file names to pd.Dataframe to match
local_files = os.listdir("/data/GZ_Decals/nomerge/")
ra, dec = [], []
for i in tqdm(range(len(local_files))):
    if ".fits" in local_files[i]:
        t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1])
        ra.append(t_ra)
        dec.append(t_dec)
local_files = os.listdir("/data/GZ_Decals/merge/")
for i in tqdm(range(len(local_files))):
    if ".fits" in local_files[i]:
        t_ra, t_dec= float(local_files[i].split("_")[0]), float(local_files[i].split("_")[1])
        ra.append(t_ra)
        dec.append(t_dec)
local_dir = pd.DataFrame(list(zip(ra, dec)), columns=["ra", "dec"])

100%|███████████████████████████████████████████████████████████████████████████| 3209/3209 [00:00<00:00, 495564.12it/s]


In [6]:
# match Main Galaxy Sample catalog, add petro_mag info
df_auto_matched = match(MGS, df_auto, 5, ["MGS_ra", "MGS_dec"])

In [7]:
df_auto_matched_2 = match(local_dir, df_auto_matched, 5, ["loc_ra", "loc_dec"])
df_auto_matched_2 = df_auto_matched_2.rename(columns={"ra": "auto_ra", "dec":"auto_dec"})

In [19]:
from sklearn.model_selection import train_test_split
threshold = 0.5
threshold2 = 0.6
merger = df_auto_matched_2.query('merging_minor_disturbance_fraction > %f '
                               '| merging_major_disturbance_fraction > %f '
                               '| merging_merger_fraction > %f '
                               % (threshold2, threshold2, threshold2))
round = df_auto_matched_2.query('smooth_or_featured_smooth_fraction >  %f '
                                '& how_rounded_round_fraction > %f '
                                '& merging_none_fraction > %f'
                                % (0.7, 0.8, threshold2))
between = df_auto_matched_2.query('smooth_or_featured_smooth_fraction >  %f '
                                  '& how_rounded_in_between_fraction > %f'
                                  '& merging_none_fraction > %f'
                                  % (0.7, 0.85, threshold2))
cigar = df_auto_matched_2.query('smooth_or_featured_smooth_fraction > %f '
                                '& how_rounded_cigar_shaped_fraction > %f'
                                '& merging_none_fraction > %f'
                                % (threshold, threshold2, threshold2))
edgeOn = df_auto_matched_2.query('smooth_or_featured_featured_or_disk_fraction > %f '
                                 '& disk_edge_on_yes_fraction > %f'
                                 '& merging_none_fraction > %f'
                            % (threshold, 0.7, threshold2))
noBar = df_auto_matched_2.query('smooth_or_featured_featured_or_disk_fraction > %f '
                                '& disk_edge_on_no_fraction > %f '
                                '& bar_no_fraction > %f '
                                '& merging_none_fraction > %f'
                                % (threshold, threshold, 0.7, threshold2))
strongBar = df_auto_matched_2.query('smooth_or_featured_featured_or_disk_fraction > %f '
                                    '& disk_edge_on_no_fraction > %f '
                                    '&bar_strong_fraction > %f '
                                    '& merging_none_fraction > %f'
                                % (threshold, threshold, threshold2, threshold2))

merger.insert(loc=7, column='label', value=0)
round.insert(loc=7, column='label', value=1)
between.insert(loc=7, column='label', value=2)
cigar.insert(loc=7, column='label', value=3)
edgeOn.insert(loc=7, column='label', value=4)
noBar.insert(loc=7, column='label', value=5)
strongBar.insert(loc=7, column='label', value=6)

merger_train, merger_test = train_test_split(merger, test_size=0.1, random_state=1926)
merger_train, merger_valid = train_test_split(merger_train, test_size=0.1, random_state=1926)
merger_train.insert(loc=7, column='func', value="train")
merger_valid.insert(loc=7, column='func', value="valid")
merger_test.insert(loc=7, column='func', value="test")

round_train, round_test = train_test_split(round, test_size=0.1, random_state=1926)
round_train, round_valid = train_test_split(round_train, test_size=0.1, random_state=1926)
round_train.insert(loc=7, column='func', value="train")
round_valid.insert(loc=7, column='func', value="valid")
round_test.insert(loc=7, column='func', value="test")

between_train, between_test = train_test_split(between, test_size=0.1, random_state=1926)
between_train, between_valid = train_test_split(between_train, test_size=0.1, random_state=1926)
between_train.insert(loc=7, column='func', value="train")
between_valid.insert(loc=7, column='func', value="valid")
between_test.insert(loc=7, column='func', value="test")

cigar_train, cigar_test = train_test_split(cigar, test_size=0.1, random_state=1926)
cigar_train, cigar_valid = train_test_split(cigar_train, test_size=0.1, random_state=1926)
cigar_train.insert(loc=7, column='func', value="train")
cigar_valid.insert(loc=7, column='func', value="valid")
cigar_test.insert(loc=7, column='func', value="test")

edgeOn_train, edgeOn_test = train_test_split(edgeOn, test_size=0.1, random_state=1926)
edgeOn_train, edgeOn_valid = train_test_split(edgeOn_train, test_size=0.1, random_state=1926)
edgeOn_train.insert(loc=7, column='func', value="train")
edgeOn_valid.insert(loc=7, column='func', value="valid")
edgeOn_test.insert(loc=7, column='func', value="test")

noBar_train, noBar_test = train_test_split(noBar, test_size=0.1, random_state=1926)
noBar_train, noBar_valid = train_test_split(noBar_train, test_size=0.1, random_state=1926)
noBar_train.insert(loc=7, column='func', value="train")
noBar_valid.insert(loc=7, column='func', value="valid")
noBar_test.insert(loc=7, column='func', value="test")

strongBar_train, strongBar_test = train_test_split(strongBar, test_size=0.1, random_state=1926)
strongBar_train, strongBar_valid = train_test_split(strongBar_train, test_size=0.1, random_state=1926)
strongBar_train.insert(loc=7, column='func', value="train")
strongBar_valid.insert(loc=7, column='func', value="valid")
strongBar_test.insert(loc=7, column='func', value="test")

merger = pd.concat([merger_train, merger_valid, merger_test])
round = pd.concat([round_train, round_valid, round_test])
between = pd.concat([between_train, between_valid, between_test])
cigar = pd.concat([cigar_train, cigar_valid, cigar_test])
edgeOn = pd.concat([edgeOn_train, edgeOn_valid, edgeOn_test])
noBar = pd.concat([noBar_train, noBar_valid, noBar_test])
strongBar = pd.concat([strongBar_train, strongBar_valid, strongBar_test])

In [26]:
len(merger), len(round), len(between), len(cigar), len(edgeOn), len(noBar), len(strongBar), len(merger)+len(round)+ len(between)+ len(cigar)+ len(edgeOn)+ len(noBar)+ len(strongBar)

(3134, 25481, 25045, 15620, 10381, 8993, 1375, 90029)

In [22]:
merger.head(1)

,loc_ra,loc_dec,MGS_ra,MGS_dec,petro_mag,auto_ra,auto_dec,func,label,iauname,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
139004,214.826545,21.409116,214.826573,21.409131,17.311718,214.826545,21.409116,train,0,J141918.37+212432.8,...,0.166326,0.166326,0.707526,0.026712,0.006131,0.006131,1.0,dr5/png/J141/J141918.37+212432.8.png,141.600783,False


In [23]:
df_auto_matched_2 = pd.concat([merger, round, between, cigar, edgeOn, noBar, strongBar])
df_auto_matched_2.to_csv("/data/renhaoye/MorCG/dataset/astre_catalog.csv")

In [24]:
df_auto_matched_2

,loc_ra,loc_dec,MGS_ra,MGS_dec,petro_mag,auto_ra,auto_dec,func,label,iauname,...,bar_proportion_volunteers_asked,bulge_size_proportion_volunteers_asked,how_rounded_proportion_volunteers_asked,edge_on_bulge_proportion_volunteers_asked,spiral_winding_proportion_volunteers_asked,spiral_arm_count_proportion_volunteers_asked,merging_proportion_volunteers_asked,file_loc,wrong_size_statistic,wrong_size_warning
139004,214.826545,21.409116,214.826573,21.409131,17.311718,214.826545,21.409116,train,0,J141918.37+212432.8,...,0.166326,0.166326,0.707526,0.026712,0.006131,0.006131,1.0,dr5/png/J141/J141918.37+212432.8.png,141.600783,False
45201,209.706484,18.423281,209.706453,18.423286,17.063713,209.706484,18.423281,train,0,J135849.54+182523.8,...,0.185039,0.185039,0.698004,0.032283,0.012590,0.012590,1.0,dr5/png/J135/J135849.54+182523.8.png,143.440521,False
250982,233.434377,7.922783,233.434343,7.922756,17.394793,233.434377,7.922783,train,0,J153344.24+075521.9,...,0.302540,0.302540,0.565423,0.016178,0.009550,0.009550,1.0,dr5/png/J153/J153344.24+075521.9.png,131.833245,False
251904,225.919110,8.917124,225.919121,8.917116,16.537960,225.919110,8.917124,train,0,J150340.16+085454.0,...,0.251689,0.251689,0.256990,0.437910,0.116010,0.116010,1.0,dr5/png/J150/J150340.16+085454.0.png,147.262206,False
78708,166.657883,28.847830,166.657877,28.847797,17.162458,166.657883,28.847830,train,0,J110637.88+285052.1,...,0.738474,0.738474,0.162567,0.029588,0.610433,0.610433,1.0,dr5/png/J110/J110637.88+285052.1.png,150.242283,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13226,239.099220,25.818360,239.099193,25.818381,15.580478,239.099220,25.818360,test,6,J155623.81+254906.1,...,0.608574,0.608574,0.201390,0.135467,0.110805,0.110805,1.0,dr5/png/J155/J155623.81+254906.1.png,150.955346,False
129091,157.419666,1.931971,157.419659,1.931970,15.101380,157.419666,1.931971,test,6,J102940.71+015555.0,...,0.872192,0.872192,0.054614,0.042407,0.851223,0.851223,1.0,dr5/png/J102/J102940.71+015555.0.png,132.039321,False
124332,125.774908,26.089137,125.774898,26.089131,17.161108,125.774908,26.089137,test,6,J082305.97+260520.8,...,0.900918,0.900918,0.035746,0.026521,0.876060,0.876060,1.0,dr5/png/J082/J082305.97+260520.8.png,152.515402,False
39719,209.303073,22.281779,209.303080,22.281784,15.812901,209.303073,22.281779,test,6,J135712.73+221654.4,...,0.609371,0.609371,0.263189,0.079461,0.090545,0.090545,1.0,dr5/png/J135/J135712.73+221654.4.png,140.403712,False


In [ ]:
# check /data/renhaoye/

In [ ]:
ROOT = "/data/renhaoye/decals_2022/"  # 根目录
sum = 0
with open("/data/renhaoye/decals_2022/dataset_txt/overlap_train.txt", "w") as w:  # 需要输入txt的特征文件名
    for i in tqdm(range(len(overlap_train))):  # 整个df的循环
        label = str(overlap_train.iloc[i].label)
        ra, dec = str(overlap_train.iloc[i].ra), str(overlap_train.iloc[i].dec),   # 先拿坐标和标签，这一步仅适用于decals的，sdss匹配的不可以
        if os.path.exists("/data/renhaoye/decals_2022/out_decals/scaled/" + ra + "_" + dec + "_0.262_grz_.fits"):
            w.write("/data/renhaoye/decals_2022/out_decals/scaled/" + ra + "_" + dec + "_0.262_grz_.fits " + str(label)[0] +"\n")
        else:
            sum += 1
print(sum)
sum = 0
with open("/data/renhaoye/decals_2022/dataset_txt/overlap_valid.txt", "w") as w:  # 需要输入txt的特征文件名
    for i in tqdm(range(len(overlap_valid))):  # 整个df的循环
        label = str(overlap_valid.iloc[i].label)
        ra, dec = str(overlap_valid.iloc[i].ra), str(overlap_valid.iloc[i].dec)   # 先拿坐标和标签，这一步仅适用于decals的，sdss匹配的不可以
        # print("/data/renhaoye/decals_2022/out_decals/scaled/" + ra + "_" + dec + "_0.262_grz_.fits " + str(label)[0] +"\n")
        if os.path.exists("/data/renhaoye/decals_2022/out_decals/scaled/" + ra + "_" + dec + "_0.262_grz_.fits"):
            w.write("/data/renhaoye/decals_2022/out_decals/scaled/" + ra + "_" + dec + "_0.262_grz_.fits " + str(label)[0] +"\n")
        else:
            sum += 1
print(sum)

In [8]:
out = os.listdir("/data/renhaoye/decals_2022/out_decals/scaled")
out_ra, out_dec = [], []
# for i in tqdm(range(10)):
for i in tqdm(range(len(out))):
    t_ra, t_dec= float(out[i].split("_")[0]), float(out[i].split("_")[1])
    out_ra.append(t_ra)
    out_dec.append(t_dec)
out_dir = pd.DataFrame(list(zip(out_ra, out_dec)), columns=["ra", "dec"])

100%|███████████████████████████████████████████████████████████████████████| 457118/457118 [00:00<00:00, 510650.89it/s]


In [9]:
out_match = match(out_dir, df_auto_matched_2.rename(columns={"MGS_ra":"ra", "MGS_dec":"dec"}), 15, ["out_ra", "out_dec"])

In [27]:
with fits.open("/data/renhaoye/MorCG/dataset/MGS_out_DECaLS.fits") as hdul:
    ra = np.array(hdul[1].data["ra      "])
    dec = np.array(hdul[1].data["dec     "])
    mzls = pd.DataFrame(np.array((ra, dec)).T, columns=["ra", "dec"])

In [28]:
a = os.listdir("/data/renhaoye/decals_2022/in_decals/fits")

In [31]:
a[1]

'195.388715_5.035083.fits'